# Classification

In [ ]:
entrainer modèle pour classifier :
    -classification personelle
- associer les phrases à des cas féminins masculins ou neutre (classes)
ensuite : effacer les noms et voir si ça marche tjr
    => test de prédiction

- prendre des sujets : ex : faire un régime
    voir les thématiques associées à des femmes
    quand je fait la détection, ne pas utiliser les noms propres différents (NOM1)

    ensuite, inverser nom féminin et masculin et tester la réussite du schéma winogrande
    1 nom féminin, masculin  ou les 2

environ 200-400 annotation
éventuellement utiliser des recherches de mots (pas trop généraux et plusieurs)

utiliser : sklearn => word embeding (BERT)

ensuite : validation croisée , couper le corpus en pluseurs morceaux et faire le test plusieurs fois, car on manque de données
=> k fold

faire des stats sur les sous ensemble d'annotation

prendre RoBerta finetuné sur winograde et lui proposer de items non biaisé à résoudre et comparer les scores avecdes items biaisés

pour bert: faire en sorte que ça donne bien les embedding

In [ ]:
import sklearn
print(sklearn.__version__)


1.2.1


In [29]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# === 1. Chargement du modèle CamemBERT ===
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModel.from_pretrained("camembert-base")

# === 2. Fonction pour encoder un texte en embedding BERT ===
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Moyenne des embeddings de tous les tokens
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# === 3. Données d'exemple ===
X_texts = [
    "Le chat est mignon", "Le chien aboie", "Le soleil brille", "Il pleut aujourd'hui", "J'aime le chocolat",
    "Le chat dort", "Le chien court", "Le ciel est bleu", "Il neige dehors", "Je mange une pomme"
]

y = [0, 0, 1, 1, 0, 0, 0, 1, 1, 0]

# === 4. Transformer les textes en embeddings ===
X_embed = [get_bert_embedding(text) for text in X_texts]
X_embed_np = np.vstack(X_embed)

# === 5. Utilisation dans scikit-learn ===
X_train, X_test, y_train, y_test = train_test_split(X_embed_np, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter = 1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy :", accuracy_score(y_test, y_pred))




Accuracy : 1.0


In [36]:
print(X_test.shape)


(2, 768)


In [38]:
get_bert_embedding("njknjzeflzfnzlfznnfzfi").reshape(1, -1).shape

(1, 768)

In [30]:
y2 = clf.predict(get_bert_embedding("njknjzeflzfnzlfznnfzfi").reshape(1, -1))
print(y2)

[1]


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=2)
kf.get_n_splits(X)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

KFold(n_splits=2, random_state=None, shuffle=False)
Fold 0:
  Train: index=[2 3]
  Test:  index=[0 1]
Fold 1:
  Train: index=[0 1]
  Test:  index=[2 3]


In [ ]:
sport
caring
wealth
advice

hesitation, question, advice
performance, sucess, competence
Failure
comparison of performance, road to sucess
relationship, helping

aussi : comptabiliser her/his pour les femmes/h

In [ ]:
# Nom du fichier source et du fichier de sortie
fichier_entree = "train.txt"
fichier_sortie = "train_cut.txt"

# Lecture du fichier et filtrage
with open(fichier_entree, "r", encoding="utf-8") as f_in:
    lignes = f_in.readlines()

# Garder une ligne sur deux
lignes_filtrees = lignes[::2]  # [::2] = toutes les 2 lignes en partant de 0

# Écriture du résultat dans un nouveau fichier
with open(fichier_sortie, "w", encoding="utf-8") as f_out:
    f_out.writelines(lignes_filtrees)

print(f"{len(lignes_filtrees)} lignes conservées dans '{fichier_sortie}'.")


1279 lignes conservées dans 'train_cut.txt'.


In [2]:
from google.colab import files
uploaded = files.upload()

# Le code va vous inviter à choisir un fichier depuis votre ordinateur
# Une fois le fichier téléchargé, il sera disponible dans le répertoire temporaire de Colab.

Saving class_2.csv to class_2.csv


In [3]:
import pandas as pd
import re

# Chargement de ton fichier CSV (si tu l'as en CSV)
df = pd.read_csv('class_2.csv')

# Exemple avec les colonnes en question
liste_2D = []

for _, row in df.iterrows():
    text = row['text']
    sentiment = row['sentiment']

    # Extraction avec regex
    qid_match = re.search(r'qID\s([^\s,]+)', text)
    sentence_match = re.search(r'sentence\s(.*?),\soption1', text)
    answer_match = re.search(r'answer\s?(\d+)', text)

    if qid_match and sentence_match and answer_match:
        qid = qid_match.group(1).strip()
        sentence = sentence_match.group(1).strip()
        answer = answer_match.group(1).strip()

        liste_2D.append([qid, sentence, answer, sentiment])



In [ ]:
print(liste_2D)

[['3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2', "Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.", '2', 'other'], ['3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1', 'He never comes to my home, but I always go to his house because the _ is smaller.', '1', 'comparison of performance, road to sucess'], ['3D5G8J4N5CI2K40F4RZLF9OG2CKVTH-2', "Kyle doesn't wear leg warmers to bed, while Logan almost always does. _ is more likely to live in a colder climate.", '2', 'other'], ['3DW3BNF1GHG4SV9KHVCQWF4L0K98VE-2', 'The GPS and map helped me navigate home.  I got lost when the _ got turned upside down.', '2', 'Failure'], ['3DTJ4WT8BFTH45NSFTZT1T1BND3ZEV-2', 'Emily looked up and saw Patricia racing by overhead, as _ was on the ramp .', '2', 'other'], ['304QEQWKZRY6EBUR5DPX06S39IH0OJ-2', 'The treasury workers took the gold bars off of the trolley and stacked them in the safe until the _ was empty.', '2', 'other'], ['36FFXPMSTB2HPB0N3LWI6YRLWWHHOX-1', 'He thought he ha

In [44]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm.auto import tqdm # Utilisation de tqdm.auto pour une détection automatique dans Colab

# --- Détection GPU/CPU et configuration du périphérique ---
# Cette partie est essentielle pour que PyTorch sache où envoyer les calculs
if torch.cuda.is_available():
    device = torch.device("cuda") # 'cuda' indique le GPU
    print(f"GPU disponible ! Utilisation de : {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu") # 'cpu' indique le CPU si pas de GPU
    print("Pas de GPU disponible, utilisation du CPU.")

print("=== Démarrage du programme ===")

# === 1. Chargement du modèle CamemBERT ===
print("1. Chargement du tokenizer et du modèle CamemBERT...")
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
# >>> MODIFICATION IMPORTANTE ICI : Déplacer le modèle sur le périphérique (GPU si disponible)
model = AutoModel.from_pretrained("camembert-base").to(device)
print("   Chargement terminé.")

# === 2. Fonction pour encoder une liste de textes en embeddings BERT par lots ===
def get_bert_embeddings_batch(texts, batch_size=64): # Augmentez le batch_size si vous avez un bon GPU Colab
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encodage des textes par lots"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True)

        # >>> MODIFICATION IMPORTANTE ICI : Déplacer les inputs (tensors) vers le même périphérique que le modèle
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        # Ramener le tenseur résultant sur le CPU AVANT de le convertir en NumPy
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# === 3. Données d'exemple ===
# Assurez-vous que votre 'liste_2D' est bien définie ici ou importée.
# Exemple fictif pour que le code soit exécutable si 'liste_2D' n'est pas définie
try:
    _ = liste_2D
except NameError:
    # Pour un exemple réaliste si 'liste_2D' n'est pas définie
    liste_2D = [
        [0, "Le succès est une question de persévérance et de travail acharné.", 0, "comparison of performance, road to sucess"],
        [0, "Le soleil brille aujourd'hui, c'est une belle journée.", 0, "autre type de texte"],
        [0, "Elle a atteint ses objectifs grâce à une performance exceptionnelle.", 0, "comparison of performance, road to sucess"],
        [0, "Les oiseaux chantent dans le jardin, c'est très agréable.", 0, "autre type de texte"],
        [0, "Sa progression sur la route du succès est inspirante.", 0, "comparison of performance, road to sucess"],
        [0, "La recette de ce gâteau est délicieuse et facile à faire.", 0, "autre type de texte"],
        [0, "Il a surpassé toutes les attentes par sa détermination.", 0, "comparison of performance, road to sucess"],
        [0, "Le livre est très intéressant, je le recommande.", 0, "autre type de texte"],
    ] * 500 # Multiplier pour avoir plus de données et voir la progression

print("2. Préparation des données d'exemple...")
X_success = [ligne[1] for ligne in liste_2D]
y = ["1" if ligne[3] == 'comparison of performance, road to sucess' else "0" for ligne in liste_2D]
print(f"   Nombre de textes à traiter : {len(X_success)}")

# === 4. Transformer les textes en embeddings ===
print("3. Début de la transformation des textes en embeddings BERT par lots (cela peut prendre du temps)...")
# >>> MODIFICATION IMPORTANTE ICI : Vous appeliez get_bert_embeddings_batch sur un seul texte, pas sur la liste
# La ligne originale: X_embed = [get_bert_embeddings_batch(text) for text in tqdm(X_success, desc="Encodage des textes")]
# Ceci est incorrect car get_bert_embeddings_batch attend une LISTE de textes.
# De plus, vous avez une tqdm imbriquée qui n'est pas nécessaire.
# Utilisez directement la fonction de batching sur votre liste X_success :
X_embed_np = get_bert_embeddings_batch(X_success, batch_size=64) # Ajustez batch_size pour votre GPU Colab
print(f"   Transformation des embeddings terminée. Forme : {X_embed_np.shape}")

# === 5. Utilisation dans scikit-learn ===
print("4. Séparation des données en ensembles d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(X_embed_np, y, test_size=0.5, random_state=42)
print("   Données séparées.")

print("5. Entraînement du modèle de régression logistique...")
# Scikit-learn (LogisticRegression) utilise le CPU par défaut, et c'est très bien car les données sont déjà sur le CPU (X_embed_np)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
print("   Modèle entraîné.")

print("6. Évaluation du modèle...")
y_pred = clf.predict(X_test)
print("Accuracy :", accuracy_score(y_test, y_pred))
print("   Évaluation terminée.")
print("=== Programme terminé ===")

GPU disponible ! Utilisation de : Tesla T4
=== Démarrage du programme ===
1. Chargement du tokenizer et du modèle CamemBERT...
   Chargement terminé.
2. Préparation des données d'exemple...
   Nombre de textes à traiter : 201
3. Début de la transformation des textes en embeddings BERT par lots (cela peut prendre du temps)...


Encodage des textes par lots:   0%|          | 0/4 [00:00<?, ?it/s]

   Transformation des embeddings terminée. Forme : (201, 768)
4. Séparation des données en ensembles d'entraînement et de test...
   Données séparées.
5. Entraînement du modèle de régression logistique...
   Modèle entraîné.
6. Évaluation du modèle...
Accuracy : 0.801980198019802
   Évaluation terminée.
=== Programme terminé ===


In [45]:
print(y_pred)

['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']


In [42]:
test = ["Albert was better than Paul in college because _ worked harder"]
test = get_bert_embeddings_batch(test)

test = test.reshape(1, -1)
print(test.shape)
test_pred = clf.predict(test)
test_pred

Encodage des textes par lots:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 768)


array(['0'], dtype='<U1')

In [ ]:
from logging import raiseExceptions
vrai positifs et négatifs
check : prendre plein des fois la même phrase et voir si c'est bien reconnu
faire un graphe de la probabilité en fonction des différentes itéraiseExceptions


est ce que c'est possible de faire dans l'autre sens ? => partir des embeddings moyens => et arriver à des catégories
à grande dimension, on peut mettre un nb exp de vecteurs orthogonaux
anisotropie => ragarder papier,
on a des connes : car faire des sommes d'objets dans tous les espaces => n'encode rien


faire un classifier pour les femmes et les hommes
voir les corrélations des dimmensions avec le genre
mettre un couche linéaire : 768 dimension => score


réunion vendredi mitn llm : rammener des questions de recherche et des hypothèses
voir le vocabulaire très genré, ex : diet
faire des permutations des prénomask

utiliser she ou he
refaire tourner


In [ ]:
print(X_success)

NameError: name 'X_success' is not defined

In [ ]:
éviter la comparaison : car c'est trop syntaxique
prendre des sujets plus concrets
bricolage
scolaire
